In [41]:
# Load dataset
import pandas as pd 
import numpy as np

Train_set = pd.read_csv('Datasets/train.csv')
print(Train_set.head())

   id  CustomerId         Surname  CreditScore Geography Gender   Age  Tenure  \
0   0    15674932  Okwudilichukwu          668    France   Male  33.0       3   
1   1    15749177   Okwudiliolisa          627    France   Male  33.0       1   
2   2    15694510           Hsueh          678    France   Male  40.0      10   
3   3    15741417             Kao          581    France   Male  34.0       2   
4   4    15766172       Chiemenam          716     Spain   Male  33.0       5   

     Balance  NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  \
0       0.00              2        1.0             0.0        181449.97   
1       0.00              2        1.0             1.0         49503.50   
2       0.00              2        1.0             0.0        184866.69   
3  148882.54              1        1.0             1.0         84560.88   
4       0.00              2        1.0             1.0         15068.83   

   Exited  
0       0  
1       0  
2       0  
3       0  
4 

In [42]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

# 1. Remove rows with missing values in any column
Train_set = Train_set.dropna()

# Label encoding the categorical features
categorical_features = ["Surname", "Geography", "Gender"]
label_encoder_X = LabelEncoder()

for feature in categorical_features:
    Train_set[feature] = label_encoder_X.fit_transform(Train_set[feature])

# Scaling numerical features
numerical_features = ["CreditScore", "Age", "Tenure", "Balance", "NumOfProducts", "EstimatedSalary"]
scaler = StandardScaler()

for feature in numerical_features:
    Train_set[feature] = scaler.fit_transform(Train_set[feature].values.reshape(-1, 1))

Train_set.drop(["id", "CustomerId"], axis=1, inplace=True)

print(Train_set.head())

   Surname  CreditScore  Geography  Gender       Age    Tenure   Balance  \
0     1924     0.144135          0       1 -0.578074 -0.719973 -0.883163   
1     1925    -0.367706          0       1 -0.578074 -1.432694 -0.883163   
2     1178     0.268974          0       1  0.211354  1.774548 -0.883163   
3     1299    -0.941966          0       1 -0.465299 -1.076334  1.486918   
4      467     0.743362          2       1 -0.578074 -0.007253 -0.883163   

   NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  Exited  
0       0.814298        1.0             0.0         1.369486       0  
1       0.814298        1.0             1.0        -1.254085       0  
2       0.814298        1.0             0.0         1.437422       0  
3      -1.013348        1.0             1.0        -0.557018       0  
4       0.814298        1.0             1.0        -1.938770       0  


In [43]:
# 2. Split dataset into training and validation sets
from sklearn.model_selection import train_test_split

X = Train_set.drop(["Exited"], axis=1).values
y = Train_set["Exited"].values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

print("X_train shape: ", X_train.shape)


X_train shape:  (132027, 11)


In [57]:
# create dataloader
import torch
from torch.utils.data import TensorDataset, DataLoader

batch_size = 32

train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
val_data = TensorDataset(torch.from_numpy(X_val), torch.from_numpy(y_val))

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size=batch_size)

In [51]:
# Build the model

import torch.nn as nn
import torch.nn.functional as F

class FeedForward(nn.Module):
    def __init__(self, num_hidden):
        super(FeedForward, self).__init__()
        self.num_hidden = num_hidden
        
        self.net = nn.Sequential(
            nn.Linear(self.num_hidden, self.num_hidden),
            nn.ReLU(),
        )
    
    def forward(self, x):
        return self.net(x)
    
class Net(nn.Module):
    def __init__(self, input_size, hidden_layers, hidden_size, output_size):
        super(Net, self).__init__()
        self.input_size = input_size
        self.hidden_layers = hidden_layers
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.fc1 = nn.Linear(self.input_size, self.hidden_size)
        self.blocks = nn.ModuleList([FeedForward(self.hidden_size) for _ in range(self.hidden_layers)])
        self.fc2 = nn.Linear(self.hidden_size, self.output_size)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        for block in self.blocks:
            x = block(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x
    

In [53]:
# Defining hyperparameters
input_size = X_train.shape[1]
hidden_layers = 3
hidden_size = 64
output_size = 1
alpha = 0.01
epochs = 1000

# Initialize the model
model = Net(input_size, hidden_layers, hidden_size, output_size)

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=alpha)

In [ ]:
# Train the model

for epoch in range(epochs):
    train_loss = 0.0

    model.train()
    
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model.forward(data.float())
        loss = criterion(output.squeeze(), target.float())
        loss.backward()
        optimizer.step()

        train_loss += loss.item()*data.size(0)

    train_loss = train_loss/len(train_loader.dataset)

    if epoch % 10 == 0:
        print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss))